Imported important libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

Reading input

In [ ]:
Train = pd.read_csv('train.csv')
Test = pd.read_csv('test.csv')
print(Train.shape)
print(Test.shape)
id_col = Test['Id']

In [ ]:
Train.head()

In [ ]:
Test.head()

Removing columns with NaN value occurences greater than 70%

In [ ]:
major_null_list = []
for i in Train:
  if(i != 'Fault'):
    X = (Train[i].isna().sum()/len(Train[i]))*100
    Y = (Test[i].isna().sum()/len(Test[i]))*100
    #print(i,X,Y)
    if(X > 70):
      major_null_list.append(i)
print(major_null_list)
Train = Train.drop(columns=major_null_list)
Test = Test.drop(columns=major_null_list)

Formatted time

In [ ]:
# Given a trial by converting entire time to seconds
# Dropping the column gave more accuracy than the normal.


# def format_time(df):
#   crash_time = df['Crash Date/Time']
#   crash_time = [ele[11:] for ele in crash_time]
#   crash_time_secs = [int(ele[0:2])*3600+int(ele[3:5])*60+int(ele[6:8]) for ele in crash_time]
#   pm = 12*60*60
#   pm_am = [pm if ele[-2:] == 'PM' else 0  for ele in crash_time]

#   time = [sum(i) for i in zip(crash_time_secs, pm_am)]
#   df['time'] = time
#   return df

# Train = format_time(Train)
# Test = format_time(Test)

Handpicked columns(detailed examination) to remove 

In [ ]:
columns_to_remove = ["Report Number","Local Case Number","Road Name","Cross-Street Name",
                     "Person ID","Vehicle ID","Vehicle Make","Vehicle Model","Location","Crash Date/Time"]


Train = Train.drop(columns=columns_to_remove,axis=1)
columns_to_remove.append('Id')
Test = Test.drop(columns=columns_to_remove,axis=1)

print(Train.shape)
print(Test.shape)


Using Categorical Imputer

In [ ]:
from sklearn_pandas import CategoricalImputer
imputer = CategoricalImputer()

for label in Train:
  Train[label] = imputer.fit_transform(Train[label])
  
for label in Test:
  Test[label] = imputer.fit_transform(Test[label])

Label Encoding

In [ ]:
column_labels = Train.columns.values
to_encode = []
for label in Train:
  col = Train[label]
  res = all([ isinstance(ele,int) or isinstance(ele,float)  for ele in col])
  if not res:
    to_encode.append(label)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in to_encode:
  Train[i] = le.fit_transform(Train[i])
  Test[i] = le.fit_transform(Test[i])

Separating Xtrain and ytrain

In [ ]:
result = 'Fault'
y_train = Train[result]
X_train = Train.drop(columns=result)

Done with preprocessing

Gradient Boosting classifier 351 estimators lr 0.06 max_depth 7


In [ ]:
# from sklearn.ensemble import GradientBoostingClassifier
# clf = GradientBoostingClassifier(n_estimators=351, learning_rate=0.06,max_depth=7).fit(X_train, y_train)
# y_pred = clf.predict(Test)
# y1 = y_pred
# res = pd.DataFrame()
# res['Id'] = id_col
# res['Fault'] = y_pred
# res.to_csv('submission_gbcbest.csv',index=False)

# res.head()
# print("Gradient Boosting")

Installing Cat Boost

In [ ]:
!pip install catboost

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from catboost import CatBoostClassifier

k_testpred = 0
k_folds = StratifiedKFold(n_splits = 5)
i = 0

for train,val in k_folds.split(Train, y_train):
  Xp_train,yp_train = X_train.loc[train],y_train.loc[train]
  Xp_val,yp_val = X_train.loc[val],y_train.loc[val]
  print("Fitting %d th fold"%(i))
  
  clf = CatBoostClassifier(
                        verbose=1000,
                        cat_features= to_encode,
                        eval_metric="Accuracy",
                        loss_function="CrossEntropy",
                        od_type="Iter",
                        od_wait=300,
                        num_boost_round=1800
            )
  clf.fit(Xp_train,yp_train, eval_set = (Xp_val, yp_val))

  print("Completed")
  k_testpred += clf.predict_proba(Test)
  i += 1

kf_final = [np.argmax(i) for i in k_testpred]
res = pd.DataFrame()
res['Id'] = id_col
res['Fault'] = kf_final
res.to_csv('submission_cbc.csv',index=False)

XG Booster

In [ ]:
# import xgboost as xgb
# xg_class = xgb.XGBClassifier(n_estimators=300,learning_rate=0.01)
# xg_class.fit(X_train,y_train)
# y_pred = xg_class.predict(Test)
# y2 = y_pred
# res = pd.DataFrame()
# res['Id'] = id_col

# res['Fault'] = y_pred
# res.to_csv('submission_25cols_xgb.csv',index=False)

# res.head()
# print("XGBoost classifier")

 RFC

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# rfc = RandomForestClassifier()
# rfc.fit(X_train,y_train)
# print(Test.shape)
# y_pred = rfc.predict(Test)
# rfcy = y_pred

# res = pd.DataFrame()
# res['Id'] = id_col

# res['Fault'] = y_pred
# res.to_csv('submission_rfc.csv',index=False)